In [1]:
!pip install transformers
!pip install datasets
!pip install tokenizer
!pip install seqeval
!pip install transformers[torch]
!pip install seqeval

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     ----------------- ------------------- 61.4/129.4 kB 656.4 kB/s eta 0:00:01
     -------------------------------------- 129.4/129.4 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.4 MB 21.4 MB/s eta 0:00:01
   ---------------- ----------------------- 3.4/8.4 MB 36.4 MB/s eta 0:00:01
   ------------------------------ -

In [7]:
!pip install datasets

You should consider upgrading via the 'c:\users\satya\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install azureml-core
!pip install --upgrade azureml-core


In [5]:
import datasets
import numpy as np 
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

c:\Users\satya\anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
conll2003=datasets.load_dataset('conll2003')

In [7]:
conll2003['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
conll2003['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [9]:
conll2003['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [10]:
tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased')

In [11]:
example_text=conll2003['train'][0]
example_text

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [12]:
tokenized_input=tokenizer(example_text['tokens'], is_split_into_words=True)

In [13]:
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenized_input['input_ids']

[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]

In [15]:
example_text['ner_tags']

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [16]:
tokenz=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

In [17]:
tokenz

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [15]:
for i, label in enumerate(example_text['ner_tags']):
    print(i,label)

0 3
1 0
2 7
3 0
4 0
5 0
6 7
7 0
8 0


In [18]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs=tokenizer(examples['tokens'],truncation=False, is_split_into_words=True)
    labels=[]
    for i , label in enumerate(examples['ner_tags']):
        word_ids=tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx=None
        label_ids=[]
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx!=previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx=word_idx
        labels.append(label_ids)
    tokenized_inputs['labels']=labels 
    return tokenized_inputs   


In [19]:
q=tokenize_and_align_labels(conll2003['train'][4:5])

In [20]:
q

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [21]:
import torch


In [22]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]),q['labels'][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
germany_________________________________ 5
'_______________________________________ 0
s_______________________________________ 0
representative__________________________ 0
to______________________________________ 0
the_____________________________________ 0
european________________________________ 3
union___________________________________ 4
'_______________________________________ 0
s_______________________________________ 0
veterinary______________________________ 0
committee_______________________________ 0
werner__________________________________ 1
z_______________________________________ 2
##wing__________________________________ 2
##mann__________________________________ 2
said____________________________________ 0
on______________________________________ 0
wednesday_______________________________ 0
consumers_______________________________ 0
should__________________________________ 0
buy_____________________________________ 0
sheep___

In [23]:
tokenized_datasets=conll2003.map(tokenize_and_align_labels, batched=True)

In [22]:
model=AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=9)

model.safetensors: 100%|██████████| 440M/440M [00:06<00:00, 71.8MB/s] 
c:\Users\satya\anaconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\satya\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForTokenClassi

In [24]:
from transformers import TrainingArguments, Trainer

In [25]:
metric=datasets.load_metric('seqeval')

C:\Users\satya\AppData\Local\Temp\ipykernel_44500\1565578648.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=datasets.load_metric('seqeval')
c:\Users\satya\anaconda3\envs\transformers\lib\site-packages\datasets\load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
example=conll2003['train'][0]
label_list=conll2003['train'].features['ner_tags'].feature.names

In [34]:
data_collator=DataCollatorForTokenClassification(tokenizer)

In [36]:
def compute_metrics(eval_preds):
    pred_logits, labels=eval_preds
    pred_logits=np.argmax(pred_logits, axis=2)
    predictions=[
        [label_list[eval_preds] for (eval_preds,l) in zip(prediction, label) if l!=-100 ]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels=[

        [label_list[l] for (eval_preds, l) in zip(predictions, label) if l!=-100]
        for prediction, label in zip(pred_logits,labels)
    ]

    results=metric.compute(predictions=predictions, references=true_labels)
    return {
        "Precision": results['overall_precision'],
        "Recall": results['overall_recall'],
        "F1": results['overall_accuracy']
    }

In [40]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [39]:
args=TrainingArguments(
    "test-ner",evaluation_strategy='epoch',
    learning_rate=0.0005,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01
)
trainer=Trainer(
    model, 
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics 

)

In [42]:
trainer.train()

 57%|█████▋    | 500/878 [18:20<14:08,  2.25s/it]

{'loss': 0.3774, 'learning_rate': 0.00021526195899772212, 'epoch': 0.57}


100%|██████████| 878/878 [33:31<00:00,  2.29s/it]

{'eval_loss': 0.14566431939601898, 'eval_Precision': 0.8255584756898817, 'eval_Recall': 0.8433829287392326, 'eval_F1': 0.9620474367324893, 'eval_runtime': 113.4954, 'eval_samples_per_second': 28.636, 'eval_steps_per_second': 1.797, 'epoch': 1.0}
{'train_runtime': 2011.2851, 'train_samples_per_second': 6.981, 'train_steps_per_second': 0.437, 'train_loss': 0.300856414308309, 'epoch': 1.0}


TrainOutput(global_step=878, training_loss=0.300856414308309, metrics={'train_runtime': 2011.2851, 'train_samples_per_second': 6.981, 'train_steps_per_second': 0.437, 'train_loss': 0.300856414308309, 'epoch': 1.0})

In [43]:
model.save_pretrained('NER_transformer_fine_tuned')

In [58]:
tokenizer.save_pretrained('NER1_tokenizer')

('NER1_tokenizer\\tokenizer_config.json',
 'NER1_tokenizer\\special_tokens_map.json',
 'NER1_tokenizer\\vocab.txt',
 'NER1_tokenizer\\added_tokens.json',
 'NER1_tokenizer\\tokenizer.json')

In [26]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [27]:
id2label={
    str(i): label for i, label in enumerate(label_list)
}

label2id={
    label: str(i) for i, label in enumerate(label_list)
}


In [31]:
label2id
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [29]:
import json
config=json.load(open('C:/Users/satya/Desktop/NER_transformer_fine_tuned/config.json'))


In [32]:
config['label2id']=label2id
config['id2label']=id2label


In [36]:
json.dump(config,open('C:/Users/satya/Desktop/NER_transformer_fine_tuned/config.json',"w"))

In [37]:
json.load(open('C:/Users/satya/Desktop/NER_transformer_fine_tuned/config.json'))

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'O',
  '1': 'B-PER',
  '2': 'I-PER',
  '3': 'B-ORG',
  '4': 'I-ORG',
  '5': 'B-LOC',
  '6': 'I-LOC',
  '7': 'B-MISC',
  '8': 'I-MISC'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'O': '0',
  'B-PER': '1',
  'I-PER': '2',
  'B-ORG': '3',
  'I-ORG': '4',
  'B-LOC': '5',
  'I-LOC': '6',
  'B-MISC': '7',
  'I-MISC': '8'},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.37.2',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [38]:
model_fine_tuned=AutoModelForTokenClassification.from_pretrained('C:/Users/satya/Desktop/NER_transformer_fine_tuned')

from transformers import pipeline

In [44]:
nlp_pipeline=pipeline('ner',model=model_fine_tuned, tokenizer=tokenizer)

In [56]:
example='Sudhanshu kumar is the founded of Ineuron'
example1='Modi is working in BJP'
example1='Microsoft windows created their software by idea that came from the window of the house'

In [58]:
nlp_pipeline(example1)

[{'entity': 'B-MISC',
  'score': 0.78552896,
  'index': 1,
  'word': 'microsoft',
  'start': 0,
  'end': 9},
 {'entity': 'I-MISC',
  'score': 0.86615276,
  'index': 2,
  'word': 'windows',
  'start': 10,
  'end': 17}]